In [17]:
from Grammar import *

FIRST function returns set of terminals that can be deducted from non-terminal that we passed to it.

First(A)
for each rule from non-terminal A:
if first symbol is terminal h => Add to set
else it is a non-terminal B -> Add First(B)

If it is non-terminal B - we get FIRST(B) and add to set, removing empty token

In [ ]:
def first_for_all(grammar: Grammar) -> dict:
    result_dict = {}

    def first(var: Var) -> set:
        if var in result_dict.keys():
            # first already calculated
            return result_dict[var]
        else:
            # calculate first
            result = set()
            has_empty = False
            for rule in grammar.variables[var]:
                # index is used in case all rule is a vanishing non-terminals
                for index, symbol in enumerate(rule):
                    if is_empty_token(symbol):
                        has_empty = True

                    if is_token(symbol):
                        result.add(symbol)
                        break  # there is a terminal so not all symbols are vanishing non-terminals
                    else:  # is_var(rule_first)
                        if symbol not in result_dict.keys():
                            result_dict[symbol] = first(symbol)

                        # will not calculate second time because rule_first now in result_dict
                        result = result.union(first(symbol))

                        if get_empty_token() not in result_dict[symbol]:
                            break
                        elif index == len(rule) - 1:
                            # each symbol is a vanish non-terminal => add empty
                            result.add(get_empty_token())

            if not has_empty and result:
                result.discard(get_empty_token())

            return result

    for var in grammar.variables.keys():
        # calculate only if it is not already calculated inside call of FIRST
        if var not in result_dict:
            result_dict[var] = first(var)

    return result_dict


In [19]:
def main():
    grammar = Grammar(
        {(0, ''), (1, '('), (1, ')'), (2, '+'), (2, '*'), (3, 'i')},
        {
            'E' : [['T', 'R']],
            'R' : [[(2, '+'), 'T', 'R'],
                   [(0, '')]],
            'T' : [['F', 'Y']],
            'Y' : [[(2, '*'), 'F', 'Y'],
                   [(0, '')]],
            'F' : [[(1, '('), 'E', (1, ')')],
                   [(3, 'i')]]
        },
        'E'
    )
    print(first_for_all(grammar))

In [20]:
if __name__ == "__main__":
    main()

{'F': {(1, '('), (3, 'i')}, 'T': {(1, '('), (3, 'i')}, 'E': {(1, '('), (3, 'i')}, 'R': {(2, '+'), (0, '')}, 'Y': {(2, '*'), (0, '')}}
